In [44]:
from pathlib import Path
import pandas as pd 
import numpy as np

In [45]:
dataset_path: Path = Path().absolute() / "Data"

In [46]:
def retrieve_data(path: Path) -> pd.DataFrame:
    """Function that returns the requested .csv file in a pandas dataframe"""
    return pd.read_csv(path)

In [47]:
# Reading in the various dataframes

customers = retrieve_data(dataset_path / "noahs-customers.csv")
orders = retrieve_data(dataset_path / "noahs-orders.csv")
order_items = retrieve_data(dataset_path / "noahs-orders_items.csv")
products = retrieve_data(dataset_path / "noahs-products.csv")

In [48]:
# converting the existing birthdate string into a pandas datetime object
customers['birthdate'] = pd.to_datetime(customers['birthdate'])

In [49]:
customers['birth_year'] = customers['birthdate'].apply(lambda x: x.year)

# 2023 is the most recent year of the Rabbit
customers['birth_rabbit'] = 2023 - customers['birth_year']
# deducing whether a customer was born in a Year of the Rabbit or not
rabbit_customers = customers[customers['birth_rabbit'] % 12 == 0]


In [50]:
# cancer zodiac runs from June 21 to July 22
# logical condition to filter for this
is_within_range = ((rabbit_customers['birthdate'].dt.month == 6) & (rabbit_customers['birthdate'].dt.day >= 21)) | ((rabbit_customers['birthdate'].dt.month == 7) & (rabbit_customers['birthdate'].dt.day <= 22))


In [51]:
rabbit_cancer_customers = rabbit_customers[is_within_range]

In [52]:
rabbit_cancer_customers.reset_index(drop = True, inplace = True)

In [53]:
rabbit_cancer_customers = rabbit_cancer_customers[['customerid', 'name', 'birthdate','phone', 'citystatezip']]

In [54]:
rabbit_cancer_customers = rabbit_cancer_customers.merge(orders[['orderid', 'customerid', 'ordered', 'total']], on = "customerid", how = "left")

In [55]:
rabbit_cancer_customers.dropna(inplace = True)

In [56]:
rabbit_cancer_customers['ordered'] = pd.to_datetime(rabbit_cancer_customers['ordered'])

In [57]:
rabbit_cancer_customers = rabbit_cancer_customers[rabbit_cancer_customers['ordered'].dt.year == 2017]

In [58]:
rabbit_cancer_customers = rabbit_cancer_customers.drop_duplicates(subset = ['name', 'birthdate', 'citystatezip'])

In [59]:
# from puzzle two, we know that Joshua Peterson lives in Jamaica, NY. It says in the problem that the rug was given to someone in the same neighborhood.
rabbit_cancer_customers[rabbit_cancer_customers['citystatezip'].str.contains('Jamaica')]

,customerid,name,birthdate,phone,citystatezip,orderid,ordered,total
281,2550,Robert Morton,1999-07-08,917-288-9635,"Jamaica, NY 11435",18146.0,2017-07-22 11:27:48,3.08
